# **Bool and Arithmetics**

[The deatailed script](https://www.mathcha.io/editor/Pvvz5UZ1t7ktL6sZJYp19sZnX9vVserJMEKhJvvMx7)

In [1]:
import random
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
from fitter import Fitter, get_common_distributions
from deap import tools, base, creator
import seaborn as sns

sys.path.append("../")
from calculus.term import *
from calculus.strategy import *
from calculus.generation import *
from calculus.generation_threading import *

from calculus_utils.drawing import *
from calculus_utils.algorithms import *
from calculus_utils.outliers_cleaning import clear_arr

from vd_env.lambda_env import LambdaEnv
from vd_env.run_greedy_policy import run_greedy_policy
from vd_env.run_e_greedy_policy import run_e_greedy_policy

## Useful combinators

In [2]:
x, y, z = Var(), Var(), Var()
x_, y_, z_ = Atom(x), Atom(y), Atom(z)
Lambda = Abstraction
App = Application

In [3]:
I = Lambda(x, x_)
K = Lambda(x, Lambda(y, x_))
K_star = Lambda(x, Lambda(y, y_))
S = Lambda(x, Lambda(y, Lambda(z, App(x_, App(z_, App(y_, z_))))))

## Fixed-point combinators

In [4]:
f = Var()
f_ = Atom(f)

Y = Lambda(f, App(Lambda(x, App(f_, App(x_, x_))), Lambda(x, App(f_, App(x_, x_)))))

## Logic

In [5]:
c = Var()
c_ = Atom(c)

TRUE = K
FALSE = K_star
ITE = Lambda(c, Lambda(x, Lambda(y, App(c_, App(x_, y_)))))

In [17]:
# term ITE TRUE x y == x
term = App(ITE, App(TRUE, App(x_, y_)))
norm_res = term.normalize(strategy=LeftmostOutermostStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[61] => (fun v[62] => (v[0] v[1])))
3


In [18]:
# term ITE FALSE x y == y
term = App(ITE, App(FALSE, App(x_, y_)))
norm_res = term.normalize(strategy=LeftmostOutermostStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[66] => (fun v[67] => (v[66] v[67])))
3


In [19]:
a, b = Var(), Var()
a_, b_ = Atom(a), Atom(b)

NOT = Lambda(a, App(ITE, App(a_, App(FALSE, TRUE))))
AND = Lambda(a, Lambda(b, App(ITE, App(a_, App(b_, a_)))))
OR = Lambda(a, Lambda(b, App(ITE, App(a_, App(a_, b_)))))

In [25]:
# term NOT TRUE == FALSE ?
term = App(NOT, TRUE)
norm_res = term.normalize(strategy=LeftmostOutermostStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[130] => (fun v[131] => (fun v[133] => v[133])))
5


In [23]:
# term NOT FALSE == TRUE ?
term = App(NOT, FALSE)
norm_res = term.normalize(strategy=LeftmostOutermostStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[110] => (fun v[111] => (v[110] v[111])))
4


## Numbers

In [26]:
s = Var()
s_ = Atom(s)

N_0 = Lambda(s, Lambda(z, z_))
N_1 = Lambda(s, Lambda(z, App(s_, App(s_, z_))))
N_2 = Lambda(s, Lambda(z, App(s_, App(s_, App(s_, z_)))))
N_3 = Lambda(s, Lambda(z, App(s_, (App(s_, App(s_, App(s_, z_)))))))
N_4 = Lambda(s, Lambda(z, App(s_, App(s_, (App(s_, App(s_, App(s_, z_))))))))
N_5 = Lambda(s, Lambda(z, App(s_, App(s_, App(s_, (App(s_, App(s_, App(s_, z_)))))))))
N_6 = Lambda(
    s, Lambda(z, App(s_, App(s_, App(s_, App(s_, (App(s_, App(s_, App(s_, z_)))))))))
)
N_7 = Lambda(
    s,
    Lambda(
        z, App(s_, App(s_, App(s_, App(s_, App(s_, (App(s_, App(s_, App(s_, z_)))))))))
    ),
)
N_8 = Lambda(
    s,
    Lambda(
        z,
        App(
            s_,
            App(
                s_, App(s_, App(s_, App(s_, App(s_, (App(s_, App(s_, App(s_, z_))))))))
            ),
        ),
    ),
)
N_9 = Lambda(
    s,
    Lambda(
        z,
        App(
            s_,
            App(
                s_,
                App(
                    s_,
                    App(s_, App(s_, App(s_, App(s_, (App(s_, App(s_, App(s_, z_)))))))),
                ),
            ),
        ),
    ),
)
N_10 = Lambda(
    s,
    Lambda(
        z,
        App(
            s_,
            App(
                s_,
                App(
                    s_,
                    App(
                        s_,
                        App(
                            s_,
                            App(s_, App(s_, App(s_, (App(s_, App(s_, App(s_, z_))))))),
                        ),
                    ),
                ),
            ),
        ),
    ),
)

## Arithmetics

In [28]:
n = Var()
n_ = Atom(n)

SUCC = Lambda(n, Lambda(x, Lambda(y, App(x_, App(n_, App(x_, y_))))))

In [29]:
# SUCC 0 == 1
term = App(SUCC, N_0)
norm_res = term.normalize(strategy=LeftmostOutermostStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[141] => (fun v[142] => (v[141] (fun v[144] => v[144]))))
2


In [30]:
# SUCC 1 == 2
term = App(SUCC, N_1)
norm_res = term.normalize(strategy=LeftmostOutermostStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[146] => (fun v[147] => (v[146] (fun v[149] => ((v[146] v[147]) ((v[146] v[147]) v[149]))))))
2


In [31]:
p = Var()
p_ = Atom(p)

ISZERO = Lambda(n, App(n_, App(Lambda(x, FALSE), TRUE)))
PAIR = Lambda(x, Lambda(y, Lambda(p, App(p_, App(x_, y_)))))
FIRST = Lambda(p, App(p_, K))
SECOND = Lambda(p, App(p_, K_star))

In [41]:
# ISZERO N_0 = TRUE
term = App(ISZERO, N_0)
norm_res = term.normalize(strategy=LeftmostOutermostStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[230] => v[230])
2


In [38]:
# ISZERO N_1 = FALSE
term = App(ISZERO, N_9)
norm_res = term.normalize(strategy=LeftmostOutermostStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[206] => (fun v[202] => v[202]))
4


In [42]:
# FIRST PAIR N_O N_1
term = App(FIRST, App(PAIR, App(N_0, N_1)))
norm_res = term.normalize(strategy=LeftmostOutermostStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[236] => (v[236] (fun v[232] => (fun v[233] => v[232]))))
5


In [43]:
# SECOND PAIR N_O N_1
term = App(SECOND, App(PAIR, App(N_0, N_1)))
norm_res = term.normalize(strategy=LeftmostOutermostStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[246] => (v[246] (fun v[242] => (fun v[243] => v[243]))))
5


In [45]:
SINC = Lambda(p, App(App(PAIR, App(SECOND, p_)), App(SUCC, App(SECOND, p_))))

# ???
PRED = Lambda(n, App(FIRST, App(n_, App(SINC, App(PAIR, App(N_0, N_0))))))

In [46]:
m = Var()
m_ = Atom(m)

SUBSTR = Lambda(n, Lambda(m, App(m_, App(PRED, n_))))

In [47]:
LE = Lambda(n, Lambda(m, App(ISZERO, App(SUBSTR, App(n_, m_)))))
EQ = Lambda(n, Lambda(m, App(AND, App(App(LE, App(n_, m_)), App(LE, App(m_, n_))))))

In [50]:
GCD = App(
    Y,
    Lambda(
        f,
        Lambda(
            n,
            Lambda(
                m,
                App(
                    ITE,
                    App(
                        OR,
                        App(
                            ISZERO,
                            App(
                                n_,
                                App(
                                    ISZERO,
                                    App(
                                        m_,
                                        App(
                                            N_0,
                                            App(
                                                ITE,
                                                App(
                                                    EQ,
                                                    App(
                                                        n_,
                                                        App(
                                                            m_,
                                                            App(
                                                                n_,
                                                                App(
                                                                    ITE,
                                                                    App(
                                                                        LE,
                                                                        App(
                                                                            n_,
                                                                            App(
                                                                                m_,
                                                                                App(
                                                                                    f_,
                                                                                    App(
                                                                                        n_,
                                                                                        App(
                                                                                            SUBSTR,
                                                                                            App(
                                                                                                m_,
                                                                                                App(
                                                                                                    n_,
                                                                                                    App(
                                                                                                        f_,
                                                                                                        App(
                                                                                                            n_,
                                                                                                            App(
                                                                                                                SUBSTR,
                                                                                                                App(
                                                                                                                    m_,
                                                                                                                    App(
                                                                                                                        n_,
                                                                                                                        App(
                                                                                                                            f_,
                                                                                                                            App(
                                                                                                                                m_,
                                                                                                                                App(
                                                                                                                                    SUBSTR,
                                                                                                                                    App(
                                                                                                                                        n_,
                                                                                                                                        m_,
                                                                                                                                    ),
                                                                                                                                ),
                                                                                                                            ),
                                                                                                                        ),
                                                                                                                    ),
                                                                                                                ),
                                                                                                            ),
                                                                                                        ),
                                                                                                    ),
                                                                                                ),
                                                                                            ),
                                                                                        ),
                                                                                    ),
                                                                                ),
                                                                            ),
                                                                        ),
                                                                    ),
                                                                ),
                                                            ),
                                                        ),
                                                    ),
                                                ),
                                            ),
                                        ),
                                    ),
                                ),
                            ),
                        ),
                    ),
                ),
            ),
        ),
    ),
)

In [ ]:
# GCD N_2 N_7
term = App(GCD, App(N_2, N_7))

In [51]:
norm_res = term.normalize(strategy=LeftmostOutermostStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[257] => (v[257] (fun v[253] => (fun v[254] => v[254]))))
5


In [52]:
norm_res = term.normalize(strategy=RightmostInnermostStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[267] => (v[267] (fun v[263] => (fun v[264] => v[264]))))
5


In [53]:
norm_res = term.normalize(strategy=LeftmostInnermostStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[277] => (v[277] (fun v[273] => (fun v[274] => v[274]))))
5


In [54]:
norm_res = term.normalize(strategy=RightmostOutermostStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[287] => (v[287] (fun v[283] => (fun v[284] => v[284]))))
5


In [55]:
norm_res = term.normalize(strategy=RandomStrategy())
print(norm_res[0])
print(norm_res[1])

(fun v[297] => (v[297] (fun v[293] => (fun v[294] => v[294]))))
5
